In [ ]:
!pip install  ctgan==0.5.0

In [ ]:
!conda install --force-reinstall --no-deps six==1.16.0 --yes

In [ ]:
!pip install ydata-synthetic==0.7.0

In [ ]:
!pip install matplotlib==3.1.3

In [ ]:
!conda install --force-reinstall --no-deps numpy==1.21.6  --yes
!conda install --force-reinstall  pandas==1.1.5 --yes
!pip install --force-reinstall  pandas==1.3.5 

In [ ]:
!python -V

In [ ]:
!pip install xgboost==1.5.0

In [ ]:
# Importing the CTGAN Synthesizer model from ctgan library
from ctgan import CTGANSynthesizer

# Importing the GAN,CGAN,DRAGAN,CRAMERGAN,WGAN Synthesizer model from ydata_synthetic library
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
from ydata_synthetic.synthesizers.regular import WGAN
from ydata_synthetic.synthesizers.regular import  VanilllaGAN,CGAN,DRAGAN,CRAMERGAN


In [ ]:
# Importing the libraries
import pandas as pd #Using for loading and saving dataset
from sklearn.utils import shuffle
import time
import numpy as np
import tensorflow as tf
import os
# Cross Validation / Splitting the dataset into the Training set and Test set / Normalization
from sklearn.model_selection import cross_val_score, KFold,StratifiedKFold

from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler

# plots and metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,classification_report,auc
from sklearn.metrics import recall_score,precision_score,precision_recall_fscore_support,roc_auc_score,roc_curve
tf.random.set_seed(0)
np.random.seed(0)
# Importing the Classifer Models
import xgboost as xgb
from IPython.display import display, HTML

from sklearn import svm
plt.rcParams["figure.figsize"] = (10,8)
import warnings
warnings.filterwarnings('ignore')
print('Libraries have been imported')

In [ ]:
!python -V

In [ ]:
#Loading the dataset and Pre-processing
def prepare_dataset(data_name,n_splits):
  num_cols=[]
  cat_cols=[]
  if data_name=='Breast':
    df = pd.read_csv("/kaggle/input/dataset4/Breast Cancer Dataset/data.csv") ###The path for dataset
    #Drop irrelevant features
    list_drop = ['id','Unnamed: 32','area_mean','radius_mean','perimeter_worst','radius_worst','area_worst','radius_se',
                 'perimeter_se','concave points_mean','compactness_worst','concave points_se','concavity_se','concavity_worst',
                 'fractal_dimension_worst','smoothness_mean','symmetry_mean']
    df.drop(list_drop,axis=1,inplace=True)
        #Encode categorical feature
    df['diagnosis'].replace(["B", "M"], [0,1], inplace = True)
    # df =df.astype({'diagnosis': np.int8}) ### fixing the datatypes for dataframe 
    num_cols = list(df.columns[ df.columns != 'diagnosis' ])
    cat_cols = ['diagnosis']
    target_label='diagnosis'
    folder_name = 'Breast Cancer Dataset'
    !mkdir 'Breast Cancer Dataset'
  elif data_name=='Lung':
    df = pd.read_excel("/kaggle/input/dataset4/Lung Cancer Dataset/cancer patient data sets.xlsx") ###The path for dataset
        #Drop irrelevant features
    list_drop = ['Patient Id']
    df.drop(list_drop,axis=1,inplace=True)
    #Encode categorical feature
    df['Level'].replace(["Low", "Medium","High"], [0,1,2], inplace = True)
    # Get Unique Count using Series.unique()
    for i  in df.columns.tolist():
      if df[i].unique().size > 6 :
        num_cols.append(i)
        print(f'Unique values count :{i} => {str(df[i].unique().size)} Added to num_cols')
      else :
        cat_cols.append(i)
        print(f'Unique values count :{i} => {str(df[i].unique().size)} Added to cat_cols')
    target_label='Level'
    folder_name = 'Lung Cancer Dataset'
    !mkdir 'Lung Cancer Dataset'

  elif data_name=='CTG':
    df = pd.read_csv("/kaggle/input/dataset4/CTG Dataset/CTG.csv") ###The path for dataset
    #Drop irrelevant features
    list_drop = ["FileName","Date","SegFile","b","e","A", "B","C", "D" ,"E", "AD", "DE" ,"LD", "FS", "SUSP","CLASS","DR"]
    df.drop(list_drop,axis=1,inplace=True)
    # process to delete all nan data
    df = df.dropna()
    df =df.astype({'NSP': int}) ### fixing the datatypes for dataframe 

    # Get Unique Count using Series.unique()
    for i  in df.columns.tolist():
      if df[i].unique().size > 6 :
        num_cols.append(i)
        print(f'Unique values count :{i} => {str(df[i].unique().size)} Added to num_cols')
      else :
        cat_cols.append(i)
        print(f'Unique values count :{i} => {str(df[i].unique().size)} Added to cat_cols')
    target_label='NSP'
    folder_name = 'CTG Dataset'
    !mkdir 'CTG Dataset'
  return df,target_label,num_cols,cat_cols,folder_name,n_splits

In [ ]:
#Read data data.csv
df,target_label,num_cols,cat_cols,folder_name ,n_splits= prepare_dataset('Breast',5)
df.sample(5)

In [ ]:
df.info()

In [ ]:
#check transparency of all columns
df.describe(include='all').T

In [ ]:
df[target_label].value_counts()

In [ ]:
def split_up(data):
    #Select all features  except target
    X_data = data.drop([target_label],axis=1)
    #Select target variable
    y_data = data[target_label]
      #### Splitting the dataset into training and testing 80%,20%
    X_train, X_test, y_train, y_test = train_test_split(X_data,y_data, 
                                                          test_size = 0.2, 
                                                          random_state = 0

                                                        )
      # Feature Scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test 

In [ ]:
def sensitivity1(y_true,y_pred):
    cm=confusion_matrix(y_true, y_pred)
    FP = cm.sum(axis=0) - np.diag(cm)  
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)
    Sensitivity = TP/(TP+FN)    
    return np.mean(Sensitivity)

def specificity1(y_true,y_pred):
    cm=confusion_matrix(y_true, y_pred)
    FP = cm.sum(axis=0) - np.diag(cm)  
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)
    Specificity = TN/(TN+FP)    
    return np.mean(Specificity)


In [ ]:
#Training & Evaluation Using Classifiers: xgboost, SVM
# Splitting the dataset into training and testing 80% ,20%
X_train, X_test, y_train, y_test = split_up(df)

In [ ]:
#XGBoost Classifier
#Train the data
start=time.time()
xgb_model = xgb.XGBClassifier(random_state=0,eval_metric='mlogloss').fit(X_train, y_train)

print('\n Training time {:.2f} millisecond'.format((time.time()-start)*1000))
print("\n Training Accuracy: {:.2f} %".format(xgb_model.score(X_train, y_train)*100))
    
# Applying k-Fold Cross Validation
skf = StratifiedKFold(n_splits=n_splits,random_state=None, shuffle=True) 
accuracies = []

for train_index, test_index in skf.split(X_train,y_train): 

    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold =  y_train.iloc[train_index], y_train.iloc[test_index]
    xgb_model.fit(X_train_fold, y_train_fold) 
    accuracies.append(xgb_model.score(X_test_fold, y_test_fold))


# accuracies = cross_val_score(xgb_model, X = X_train, y = y_train, cv=KFold(n_splits=10, random_state=None, shuffle=True))
print("\n Accuracy for Cross Validation: {:.2f} %".format(np.array(accuracies).mean()*100))

In [ ]:
### Evaluation
#making prediction
start=time.time()
predicted= xgb_model.predict(X_test)
print('\n Testing time {:.2f} millisecond'.format((time.time()-start)*1000))

test_time =(time.time()-start)*1000

 #calculating accuracy
print(' Test accuracy is {:.2f}%'.format(accuracy_score(y_test,predicted)*100))
report = classification_report(y_test, predicted)
print(report)


# Making the Confusion Matrix
cm = confusion_matrix(y_test, predicted)
print("\n Confusion Matrix:")
sns.heatmap(cm,annot=True,fmt='g', cmap='Blues')

In [ ]:
print("\n Precision Score: {:.2f}%".format(precision_score(y_test, predicted, average='weighted') * 100))  ##Precision = TP/(TP+FP)
print(" Recall Score: {:.2f}%".format(recall_score(y_test, predicted, average='weighted') * 100))   ##Recall= TP / (TP + FN)
print(" F1 Score: {:.2f}%".format(f1_score(y_test, predicted, average='weighted') * 100)) #### F1 = 2*[Recall*Precision]/(Recall+Precision)


if target_label =='diagnosis':
    sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])  ##TP/(TP+FN)
    specificity = cm[1,1]/(cm[1,0]+cm[1,1]) ##TN/(TN+FP)
else :
    sensitivity= sensitivity1(y_test, predicted)
    specificity=  specificity1(y_test, predicted)

print('\n Sensitivity Score: {:.2f}%'.format(sensitivity*100))
print(' Specificity Score: {:.2f}%'.format(specificity*100))

In [ ]:
#SVM Classifier
#Train the data
start=time.time()
svm_model = svm.SVC(random_state=0).fit(X_train, y_train)
print('\n Training time {:.2f} millisecond'.format((time.time()-start)*1000))

print("\n Training Accuracy: {:.2f} %".format(svm_model.score(X_train, y_train)*100))
    
# Applying k-Fold Cross Validation
skf = StratifiedKFold(n_splits=n_splits,random_state=None, shuffle=True) 
accuracies = []

for train_index, test_index in skf.split(X_train,y_train): 

    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold =  y_train.iloc[train_index], y_train.iloc[test_index]
    svm_model.fit(X_train_fold, y_train_fold) 
    accuracies.append(svm_model.score(X_test_fold, y_test_fold))


# accuracies = cross_val_score(svm_model, X = X_train, y = y_train, cv=KFold(n_splits=5, random_state=None, shuffle=True))
print("\n Accuracy for Cross Validation: {:.2f} %".format(np.array(accuracies).mean()*100))

In [ ]:
### Evaluation
#making prediction
start=time.time()
predicted= svm_model.predict(X_test)
print('\n Testing time {:.2f} millisecond'.format((time.time()-start)*1000)) 


 #calculating accuracy
print(' Test accuracy is {:.2f}%'.format(accuracy_score(y_test,predicted)*100))
report = classification_report(y_test, predicted)
print(report)


# Making the Confusion Matrix
cm = confusion_matrix(y_test, predicted)
print("\n Confusion Matrix:")
sns.heatmap(cm,annot=True,fmt='g', cmap='Blues')

In [ ]:
print("\n Precision Score: {:.2f}%".format(precision_score(y_test, predicted, average='weighted') * 100))  ##Precision = TP/(TP+FP)
print(" Recall Score: {:.2f}%".format(recall_score(y_test, predicted, average='weighted') * 100))   ##Recall= TP / (TP + FN)
print(" F1 Score: {:.2f}%".format(f1_score(y_test, predicted, average='weighted') * 100)) #### F1 = 2*[Recall*Precision]/(Recall+Precision)
if target_label =='diagnosis':
    sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])  ##TP/(TP+FN)
    specificity = cm[1,1]/(cm[1,0]+cm[1,1]) ##TN/(TN+FP)
else :
    sensitivity= sensitivity1(y_test, predicted)
    specificity=  specificity1(y_test, predicted)

print('\n Sensitivity Score: {:.2f}%'.format(sensitivity*100))
print(' Specificity Score: {:.2f}%'.format(specificity*100))

In [ ]:
def generate_results_dictionary(model):
    return {
        
   
    'GAN Methods':['Classification Model','Dataset','Cross-validation Value','Accuracy Rate','F1Score','Recall /Sensibility','Specificity','Average Time'],
        
    'Original':[model,'Original',f'CV={n_splits}'],
    'GAN':[model,'Synthetic',f'CV={n_splits}'],
    'CGAN':[model,'Synthetic',f'CV={n_splits}'],
    'CTGAN':[model,'Synthetic',f'CV={n_splits}'],
    'CRAMERGAN':[model,'Synthetic',f'CV={n_splits}'],
    'DRAGAN':[model,'Synthetic',f'CV={n_splits}'],
    'WGAN':[model,'Synthetic',f'CV={n_splits}'],
    'GAN_Mixed':[model,'Mixed',f'CV={n_splits}'],
    'CGAN_Mixed':[model,'Mixed',f'CV={n_splits}'],
    'CTGAN_Mixed':[model,'Mixed',f'CV={n_splits}'],
    'CRAMERGAN_Mixed':[model,'Mixed',f'CV={n_splits}'],
    'DRAGAN_Mixed':[model,'Mixed',f'CV={n_splits}'],
   'WGAN_Mixed':[model,'Mixed',f'CV={n_splits}'],

}

In [ ]:
Results_xgb = generate_results_dictionary("XGBoost")

In [ ]:
Results_svm= generate_results_dictionary("SVM")

In [ ]:
def train_evaulate_xgb (X_train, X_test, y_train, y_test,df_type=''):

    ##Train the data
    start=time.time()
    xgb_model = xgb.XGBClassifier(random_state=0,eval_metric='mlogloss').fit(X_train, y_train)

    print("\n Training Accuracy: {:.2f} %".format(xgb_model.score(X_train, y_train)*100))
      
  # Applying k-Fold Cross Validation
    skf = StratifiedKFold(n_splits=n_splits,random_state=None, shuffle=True) 
    accuracies = []

    for train_index, test_index in skf.split(X_train,y_train): 

        X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
        y_train_fold, y_test_fold =  y_train.iloc[train_index], y_train.iloc[test_index]
        xgb_model.fit(X_train_fold, y_train_fold) 
        accuracies.append(xgb_model.score(X_test_fold, y_test_fold))


    print("\n Accuracy for Cross Validation: {:.2f} %".format(np.array(accuracies).mean()*100))
  ### Evaluation
  #making prediction
#   start=time.time()
    predicted= xgb_model.predict(X_test)

    average_time =((time.time()-start)*1000)/2

  #calculating accuracy
    print(' Test accuracy is {:.2f}%'.format(accuracy_score(y_test,predicted)*100))
    report = classification_report(y_test, predicted)


  # Making the Confusion Matrix
    cm = confusion_matrix(y_test, predicted)
    print("\n Confusion Matrix:")
    sns.heatmap(cm,annot=True,fmt='g', cmap='Blues')
    plt.show()

    print("\n Precision Score: {:.2f}%".format(precision_score(y_test, predicted, average='weighted') * 100))  ##Precision = TP/(TP+FP)
    print(" Recall Score: {:.2f}%".format(recall_score(y_test, predicted, average='weighted') * 100))   ##Recall= TP / (TP + FN)
    print(" F1 Score: {:.2f}%".format(f1_score(y_test, predicted, average='weighted') * 100)) #### F1 = 2*[Recall*Precision]/(Recall+Precision)

    if target_label =='diagnosis':
        sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])  ##TP/(TP+FN)
        specificity = cm[1,1]/(cm[1,0]+cm[1,1]) ##TN/(TN+FP)
    else :
        sensitivity= sensitivity1(y_test, predicted)
        specificity=  specificity1(y_test, predicted)

    print('\n Sensitivity Score: {:.2f}%'.format(sensitivity*100))
    print(' Specificity Score: {:.2f}%'.format(specificity*100))

    accuracy =accuracy_score(y_test,predicted)*100
    f1=f1_score(y_test, predicted, average='weighted') * 100
    recall=recall_score(y_test, predicted, average='weighted') * 100
    precision=precision_score(y_test, predicted, average='weighted') * 100
    support =len(X_test)

    
    Results_xgb[df_type].append(round(accuracy,2))
    Results_xgb[df_type].append(round(f1,2))
    Results_xgb[df_type].append(round(recall,2))

    Results_xgb[df_type].append(round(specificity*100,2))
    Results_xgb[df_type].append(average_time)



In [ ]:
def train_evaulate_svm (X_train, X_test, y_train, y_test,df_type=''):

      ##Train the data
    start=time.time()
    svm_model = svm.SVC(random_state=0).fit(X_train, y_train)

    print("\n Training Accuracy: {:.2f} %".format(svm_model.score(X_train, y_train)*100))

        # Applying k-Fold Cross Validation
    skf = StratifiedKFold(n_splits=n_splits,random_state=None, shuffle=True) 
    accuracies = []

    for train_index, test_index in skf.split(X_train,y_train): 

        X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
        y_train_fold, y_test_fold =  y_train.iloc[train_index], y_train.iloc[test_index]
        svm_model.fit(X_train_fold, y_train_fold) 
        accuracies.append(svm_model.score(X_test_fold, y_test_fold))


    print("\n Accuracy for Cross Validation: {:.2f} %".format(np.array(accuracies).mean()*100))
      ### Evaluation
      #making prediction

    predicted= svm_model.predict(X_test)

    average_time =(time.time()-start)*1000
      #calculating accuracy
    print(' Test accuracy is {:.2f}%'.format(accuracy_score(y_test,predicted)*100))
    report = classification_report(y_test, predicted)
    print(report)


      # Making the Confusion Matrix
    cm = confusion_matrix(y_test, predicted)
    print("\n Confusion Matrix:")
    sns.heatmap(cm,annot=True,fmt='g', cmap='Blues')
    plt.show()
    print("\n Precision Score: {:.2f}%".format(precision_score(y_test, predicted, average='weighted') * 100))  ##Precision = TP/(TP+FP)
    print(" Recall Score: {:.2f}%".format(recall_score(y_test, predicted, average='weighted') * 100))   ##Recall= TP / (TP + FN)
    print(" F1 Score: {:.2f}%".format(f1_score(y_test, predicted, average='weighted') * 100)) #### F1 = 2*[Recall*Precision]/(Recall+Precision)

    if target_label =='diagnosis':
        sensitivity = cm[0,0]/(cm[0,0]+cm[0,1])  ##TP/(TP+FN)
        specificity = cm[1,1]/(cm[1,0]+cm[1,1]) ##TN/(TN+FP)
    else :
        sensitivity= sensitivity1(y_test, predicted)
        specificity=  specificity1(y_test, predicted)
    accuracy =accuracy_score(y_test,predicted)*100
    f1=f1_score(y_test, predicted, average='weighted') * 100
    recall=recall_score(y_test, predicted, average='weighted') * 100
    precision=precision_score(y_test, predicted, average='weighted') * 100
    support =len(X_test)
    Results_svm[df_type].append(round(accuracy,2))
    
    Results_svm[df_type].append(round(f1,2))
    Results_svm[df_type].append(round(recall,2))
#     Results_svm[df_type].append(round(precision,2))
#     Results_svm[df_type].append(support)
    Results_svm[df_type].append(round(specificity*100,2))
    Results_svm[df_type].append(average_time)
#     Results_svm[df_type].append(round(np.array(accuracies).mean()*100,2))

In [ ]:
# lets plot histogram of all plots
def plot_histogram(df,df_gan):
  
  all_col = df.select_dtypes(include=np.number).columns.tolist()
  
  for i in range(len(all_col)):
      plt.subplot(2,2,1)
      plt.hist(df[all_col[i]])
      plt.tight_layout()
      plt.title(all_col[i],fontsize=25)

      plt.subplot(2,2,2)
      plt.hist(df_gan[all_col[i]],color ='grey')

      plt.tight_layout()
      plt.title(f'Synthetic {all_col[i]}',fontsize=25)
      plt.show()

In [ ]:
#Define training parameters
test_size = 212 
noise_dim = 256
dim = 128
batch_size = 128
log_step = 500
learning_rate =  5e-6  
beta_1 = 0.5
beta_2 = 0.9
models_dir = './cache'
gan_args = ModelParameters(batch_size=batch_size,
                          lr=learning_rate,
                          betas=(beta_1, beta_2),
                          noise_dim=noise_dim,
                          layers_dim=dim)


In [ ]:

def drop_some_data(df,df_gan):
  
    for i in df[target_label].value_counts().items():
        drop_examples_number = df_gan[target_label].value_counts()[i[0]]- i[1]
        drop_indices = np.random.choice(df_gan.loc[df_gan[target_label] == i[0]].index, drop_examples_number, replace=False)
        df_gan.drop(drop_indices,inplace= True)


In [ ]:
def mixed_df(df,df_gan):
    mixed_wgan = pd.concat([df, df_gan])
    return shuffle(mixed_wgan)


In [ ]:
def generate_wgan_data(df,num_cols,cat_cols):
    wgan_model = WGAN
  # WGAN training


  #Training the WGAN model
    wgan = wgan_model(gan_args, n_critic=10)

    wgan.train(data = df, train_arguments=train_args, num_cols=num_cols, cat_cols=cat_cols)

    df_wgan = wgan.sample(3*len(df)).abs()
    drop_some_data(df,df_wgan)
    mixed_wgan = mixed_df(df,df_wgan)
    
    return df_wgan, mixed_wgan


In [ ]:
def generate_gan_data(df,num_cols,cat_cols):
  ### Calling the GAN model
    gan_model = VanilllaGAN

    #Training the WGAN model
    gan = gan_model(gan_args)
    gan.train(data = df, train_arguments=train_args, num_cols=num_cols, cat_cols=cat_cols)
    df_gan = gan.sample(4*len(df)).abs()
    drop_some_data(df,df_gan)
    

    
    return df_gan, mixed_df(df,df_gan)




In [ ]:
def generate_cgan_data(df,num_cols,cat_cols,label_col,epoch):
    if target_label =='diagnosis':
        train_args = TrainParameters(epochs=epoch,
                              cache_prefix='',
                              sample_interval=log_step,
               
                              labels=(0,1))
        num_classes=2
    elif target_label =='Level':
        train_args = TrainParameters(epochs=epoch,
                                cache_prefix='',
                                sample_interval=log_step,
                    
                                labels=(0,1,2))
        num_classes=3
    elif target_label =='NSP':
        train_args = TrainParameters(epochs=epoch,
                                cache_prefix='',
                                sample_interval=log_step,
              
                                labels=(0,1,2,3))
  
        num_classes=4
      ### Calling the GAN model
    cgan_model = CGAN
      # CGAN training
    cgan = cgan_model(gan_args,num_classes=num_classes)
    cgan.train(data = df, train_arguments=train_args, num_cols=num_cols, cat_cols=cat_cols,label_col=label_col)


    l1= []
    for i in df[target_label].value_counts().items():
        cond_array = np.array([i[0]])
        l1.append(cgan.sample(n_samples=i[1],condition=cond_array))
    df_cgan = shuffle(pd.concat(l1))
    return df_cgan, mixed_df(df,df_cgan)



In [ ]:
def generate_dragan_data(df,num_cols,cat_cols):
  ### Calling the DRAGAN model
  dragan_model = DRAGAN

  #Training the draCGAN model
  dragan = dragan_model(gan_args,n_discriminator=2)

  dragan.train(data = df, train_arguments=train_args, num_cols=num_cols, cat_cols=cat_cols)
  #Sampling the data

  df_dragan = dragan.sample(5*len(df)).abs()
  drop_some_data(df,df_dragan)
  return df_dragan, mixed_df(df,df_dragan)


In [ ]:
def generate_cramergan_data(df,num_cols,cat_cols):
  ### Calling the CRAMERGAN model
  cramergan_model = CRAMERGAN
  # CRAMERGAN training

  #Training the cramer model
  cramergan = cramergan_model(gan_args)

  cramergan.train(data = df, train_arguments=train_args, num_cols=num_cols, cat_cols=cat_cols)
  df_cramergan = cramergan.sample(5*len(df)).abs()
  drop_some_data(df,df_cramergan)
  return df_cramergan, mixed_df(df,df_cramergan)




In [ ]:
def generate_ctgan_data(df,cat_cols,epoch):
  ctgan = CTGANSynthesizer(epochs=epoch,
        batch_size=100,
        generator_dim=(256, 256, 256),
        discriminator_dim=(256, 256, 256))
  
  ctgan.fit(df,cat_cols)
  # We call model’s sample function to generate samples based on the learned model.
  df_ctgan = ctgan.sample(4*len(df))
  drop_some_data(df,df_ctgan)
  return df_ctgan, mixed_df(df,df_ctgan)
  

In [ ]:
def highlighting_mean_greater(s):
    return['background-color:#F7F7F7' if i%2==0 else 'background-color: ##FFDDDD' for (i,j) in enumerate(s)]
def save_results(Results_svm,Results_xgb,folder_name,n_splits,num_epochs):
    R_svm = pd.DataFrame(Results_svm)
    R_xgb = pd.DataFrame(Results_xgb)
    
    outFile = f'/kaggle/working/{folder_name}/Results_{folder_name.split()[0]}_CV={n_splits}.xlsx'
    
    if(os.path.exists(outFile)):
        with pd.ExcelWriter(outFile ,mode="a", engine="openpyxl") as writer:
            R_svm.T.style.apply(highlighting_mean_greater).to_excel(writer, engine='openpyxl', index=False, sheet_name=f'num-epochs={num_epochs}',startcol=7,startrow=6)
        with pd.ExcelWriter(outFile ,mode="a", engine="openpyxl") as writer:
            R_xgb.T.style.apply(highlighting_mean_greater).to_excel(writer, engine='openpyxl', index=False, sheet_name=f'num-epochs={num_epochs}xgb',startcol=17,startrow=6)

    else:
        with pd.ExcelWriter(outFile,engine="openpyxl") as writer:
            R_svm.T.style.apply(highlighting_mean_greater).to_excel(writer, engine='openpyxl', index=False, sheet_name=f'num-epochs={num_epochs}',startcol=7,startrow=6)
            R_xgb.T.style.apply(highlighting_mean_greater).to_excel(writer, engine='openpyxl', index=False, sheet_name=f'num-epochs={num_epochs}',startcol=17,startrow=6)
    
    print("Data appended successfully.")


In [ ]:
Results_svm= generate_results_dictionary("SVM")
Results_xgb = generate_results_dictionary("XGBoost")
train_evaulate_xgb (X_train, X_test, y_train, y_test,df_type='Original')
train_evaulate_svm (X_train, X_test, y_train, y_test,df_type='Original')
Results_svm_Original=Results_svm['Original']
Results_xgb_Original=Results_xgb['Original']

In [ ]:
epochs = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

for epoch in epochs:
    Results_svm= generate_results_dictionary("SVM")
    Results_xgb = generate_results_dictionary("XGBoost")
    X_train, X_test, y_train, y_test = split_up(df)    

    Results_svm['Original']=Results_svm_Original
    Results_xgb['Original']=Results_xgb_Original
    train_args = TrainParameters(epochs=epoch,
                            sample_interval=log_step)
    for Gmodel in  ['WGAN','GAN','CGAN','DRAGAN','CRAMERGAN','CTGAN']:
        display(HTML('<h1Working With{} Epcochs In {}</h1>'.format(epoch,Gmodel)))    
        if Gmodel=='WGAN':
            working_df,mixed_working_df = generate_wgan_data(df,num_cols,cat_cols)
        elif  Gmodel=='GAN':
            working_df,mixed_working_df = generate_gan_data(df,num_cols,cat_cols)
        elif  Gmodel=='CGAN':
            if target_label =='Level':
                working_df,mixed_working_df = generate_cgan_data(df,num_cols,['Gender'],target_label,epoch)
            elif target_label =='NSP':
                working_df,mixed_working_df = generate_cgan_data(df,num_cols,['Tendency', 'DS', 'DP'],target_label,epoch)
            else :
                working_df,mixed_working_df = generate_cgan_data(df,num_cols,[],target_label,epoch)
        elif  Gmodel=='DRAGAN':
            working_df,mixed_working_df = generate_cramergan_data(df,num_cols,cat_cols)
        elif  Gmodel=='CRAMERGAN':
            working_df,mixed_working_df = generate_dragan_data(df,num_cols,cat_cols)
        elif  Gmodel=='CTGAN':
            working_df,mixed_working_df = generate_ctgan_data(df,cat_cols,epoch)
            
#         plot_histogram(df,working_df)   
#         plot_histogram(df,mixed_working_df)   
#         print(working_df[target_label].value_counts())


        working_df.to_csv(f'/kaggle/working/{folder_name}/df_{Gmodel}_{epoch}.csv')
        mixed_working_df.to_csv(f'/kaggle/working/{folder_name}/mixed_{Gmodel}_{epoch}.csv') 

        X_train, X_test, y_train, y_test = split_up(working_df)
        train_evaulate_xgb (X_train, X_test, y_train, y_test,df_type=Gmodel)
        train_evaulate_svm (X_train, X_test, y_train, y_test,df_type=Gmodel)   

        X_train, X_test, y_train, y_test = split_up(mixed_working_df)
        train_evaulate_xgb (X_train, X_test, y_train, y_test,df_type=f'{Gmodel}_Mixed')
        train_evaulate_svm (X_train, X_test, y_train, y_test,df_type=f'{Gmodel}_Mixed') 
        
    save_results(Results_svm,Results_xgb,folder_name,n_splits,epoch)
#     save_results(Results_xgb,'xgb',folder_name,n_splits,epoch)


In [ ]:
Results_svm